<a href="https://colab.research.google.com/github/lain-93/Machine-Learning/blob/main/vanillaGAN_MIMIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
from google.colab import files
uploaded = files.upload()

Saving CALLOUT.csv to CALLOUT (2).csv


In [3]:
import pandas as pd

df = pd.read_csv('CALLOUT.csv')
print(df.columns)


Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SUBMIT_WARDID', 'SUBMIT_CAREUNIT',
       'CURR_WARDID', 'CURR_CAREUNIT', 'CALLOUT_WARDID', 'CALLOUT_SERVICE',
       'REQUEST_TELE', 'REQUEST_RESP', 'REQUEST_CDIFF', 'REQUEST_MRSA',
       'REQUEST_VRE', 'CALLOUT_STATUS', 'CALLOUT_OUTCOME', 'DISCHARGE_WARDID',
       'ACKNOWLEDGE_STATUS', 'CREATETIME', 'UPDATETIME', 'ACKNOWLEDGETIME',
       'OUTCOMETIME', 'FIRSTRESERVATIONTIME', 'CURRENTRESERVATIONTIME'],
      dtype='object')


In [4]:
file_name = "CALLOUT.csv"
columns_to_drop = ['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SUBMIT_CAREUNIT',
       'REQUEST_TELE', 'REQUEST_RESP', 'REQUEST_CDIFF', 'REQUEST_MRSA',
       'REQUEST_VRE', 'CALLOUT_STATUS','DISCHARGE_WARDID', 'CREATETIME', 'UPDATETIME', 'ACKNOWLEDGETIME',
       'OUTCOMETIME', 'FIRSTRESERVATIONTIME', 'CURRENTRESERVATIONTIME','CALLOUT_WARDID']
categorical_features = ['CURR_CAREUNIT','CALLOUT_SERVICE', 'CALLOUT_OUTCOME', 'ACKNOWLEDGE_STATUS']
continuous_features = ['SUBMIT_WARDID', 'CURR_WARDID']

# training configuration
noise_dim = 6
dim = 6
batch_size = 2

log_step = 100
epochs = 5000+1
learning_rate = 5e-4

In [5]:
df['CURR_WARDID'].unique()

array([29., 55., 17.,  3., 33.,  8.,  2., 45., 54., 15.,  4., 36., 18.,
       23., 31., 40., 57.,  7., 24., 14., 52., 28., 49., 50., 48., 12.,
       25., 13., 32.,  9., 44., 27., 39., 51., nan, 41.])

In [6]:
df = pd.read_csv(file_name)
df.drop(columns_to_drop, axis=1, inplace=True)
print(df.isnull().sum())

df.dropna(inplace=True)

print(df.columns)
print(df.isnull().sum())

SUBMIT_WARDID         1
CURR_WARDID           1
CURR_CAREUNIT         1
CALLOUT_SERVICE       0
CALLOUT_OUTCOME       0
ACKNOWLEDGE_STATUS    0
dtype: int64
Index(['SUBMIT_WARDID', 'CURR_WARDID', 'CURR_CAREUNIT', 'CALLOUT_SERVICE',
       'CALLOUT_OUTCOME', 'ACKNOWLEDGE_STATUS'],
      dtype='object')
SUBMIT_WARDID         0
CURR_WARDID           0
CURR_CAREUNIT         0
CALLOUT_SERVICE       0
CALLOUT_OUTCOME       0
ACKNOWLEDGE_STATUS    0
dtype: int64


In [7]:
#encoding categorical columns (not one hot encoding since it is not required for GANs)

for column in categorical_features:
  df[column] = df[column].astype('category').cat.codes

df.head()

,SUBMIT_WARDID,CURR_WARDID,CURR_CAREUNIT,CALLOUT_SERVICE,CALLOUT_OUTCOME,ACKNOWLEDGE_STATUS
0,52.0,29.0,2,7,1,0
1,15.0,55.0,1,2,1,0
2,12.0,55.0,1,2,1,0
3,7.0,17.0,0,0,1,0
4,57.0,3.0,3,11,1,0


In [8]:
#encoding all continious features to equally sized bins

import numpy as np

for column in continuous_features:
  min = df[column].min()
  print(min)
  max = df[column].max()
  print(max)
  feature_bins = pd.cut(df[column], bins=np.linspace(min, max, 25), labels=False)
  df.drop([column], axis=1, inplace=True)
  df = pd.concat([df, feature_bins], axis=1)
  print(df)

7.0
57.0
       CURR_WARDID  CURR_CAREUNIT  ...  ACKNOWLEDGE_STATUS  SUBMIT_WARDID
0             29.0              2  ...                   0           21.0
1             55.0              1  ...                   0            3.0
2             55.0              1  ...                   0            2.0
3             17.0              0  ...                   0            NaN
4              3.0              3  ...                   0           23.0
...            ...            ...  ...                 ...            ...
34494         54.0              2  ...                   0            7.0
34495         45.0              2  ...                   0            7.0
34496         45.0              2  ...                   0            7.0
34497         45.0              2  ...                   0           20.0
34498         23.0              2  ...                   0            7.0

[34498 rows x 6 columns]
2.0
57.0
       CURR_CAREUNIT  CALLOUT_SERVICE  ...  SUBMIT_WARDID  CURR_WARD

In [9]:
from sklearn.preprocessing import PowerTransformer


pw= PowerTransformer(method='yeo-johnson', standardize=True, copy=True)
pwt=pw.fit_transform(df[df.columns])

print(df)

       CURR_CAREUNIT  CALLOUT_SERVICE  ...  SUBMIT_WARDID  CURR_WARDID
0                  2                7  ...           21.0         11.0
1                  1                2  ...            3.0         23.0
2                  1                2  ...            2.0         23.0
3                  0                0  ...            NaN          6.0
4                  3               11  ...           23.0          0.0
...              ...              ...  ...            ...          ...
34494              2                7  ...            7.0         22.0
34495              2                7  ...            7.0         18.0
34496              2                7  ...            7.0         18.0
34497              2                7  ...           20.0         18.0
34498              2                7  ...            7.0          9.0

[34498 rows x 6 columns]


In [10]:
df[df.columns]=pwt
df

,CURR_CAREUNIT,CALLOUT_SERVICE,CALLOUT_OUTCOME,ACKNOWLEDGE_STATUS,SUBMIT_WARDID,CURR_WARDID
0,-0.027570,-0.057170,0.388992,-0.234006,1.037869,-0.216928
1,-0.904566,-1.141645,0.388992,-0.234006,-1.181386,1.321851
2,-0.904566,-1.141645,0.388992,-0.234006,-1.442107,1.321851
3,-1.802521,-1.828948,0.388992,-0.234006,NaN,-0.916680
4,0.836087,0.592941,0.388992,-0.234006,1.190486,-1.879437
...,...,...,...,...,...,...
34494,-0.027570,-0.057170,0.388992,-0.234006,-0.434360,1.198647
34495,-0.027570,-0.057170,0.388992,-0.234006,-0.434360,0.697970
34496,-0.027570,-0.057170,0.388992,-0.234006,-0.434360,0.697970
34497,-0.027570,-0.057170,0.388992,-0.234006,0.958156,0.697970


In [11]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import Model

from tensorflow.keras.optimizers import Adam

class GAN():
    
    def __init__(self, gan_args):
        [self.batch_size, lr, self.noise_dim,
         self.data_dim, layers_dim] = gan_args

        self.generator = Generator(self.batch_size).\
            build_model(input_shape=(self.noise_dim,), dim=layers_dim, data_dim=self.data_dim)

        self.discriminator = Discriminator(self.batch_size).\
            build_model(input_shape=(self.data_dim,), dim=layers_dim)

        optimizer = Adam(lr, 0.5)

        # Build and compile the discriminator
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.noise_dim,))
        record = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(record)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def get_data_batch(self, train, batch_size, seed=0):
        # # random sampling - some samples will have excessively low or high sampling, but easy to implement
        # np.random.seed(seed)
        # x = train.loc[ np.random.choice(train.index, batch_size) ].values
        # iterate through shuffled indices, so every sample gets covered evenly

        start_i = (batch_size * seed) % len(train)
        stop_i = start_i + batch_size
        shuffle_seed = (batch_size * seed) // len(train)
        np.random.seed(shuffle_seed)
        train_ix = np.random.choice(list(train.index), replace=False, size=len(train))  # wasteful to shuffle every time
        train_ix = list(train_ix) + list(train_ix)  # duplicate to cover ranges past the end of the set
        x = train.loc[train_ix[start_i: stop_i]].values
        return np.reshape(x, (batch_size, -1))
        
    def train(self, data, train_arguments):
        [cache_prefix, epochs, sample_interval] = train_arguments
        
        data_cols = data.columns

        # Adversarial ground truths
        valid = np.ones((self.batch_size, 1))
        fake = np.zeros((self.batch_size, 1))

        for epoch in range(epochs):    
            # ---------------------
            #  Train Discriminator
            # ---------------------
            batch_data = self.get_data_batch(data, self.batch_size)
            noise = tf.random.normal((self.batch_size, self.noise_dim))

            # Generate a batch of new images
            gen_data = self.generator.predict(noise)
    
            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(batch_data, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_data, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
            # ---------------------
            #  Train Generator
            # ---------------------
            noise = tf.random.normal((self.batch_size, self.noise_dim))
            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)
    
            # Plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
    
            # If at save interval => save generated events
            if epoch % sample_interval == 0:
                #Test here data generation step
                # save model checkpoints
                model_checkpoint_base_name = 'model/' + cache_prefix + '_{}_model_weights_step_{}.h5'
                self.generator.save_weights(model_checkpoint_base_name.format('generator', epoch))
                self.discriminator.save_weights(model_checkpoint_base_name.format('discriminator', epoch))

                #Here is generating the data
                z = tf.random.normal((6, self.noise_dim))
                print('*********************************************')
                print(z)
                print('*********************************************')
                gen_data = self.generator(z)
                print('generated_data')
                print(gen_data)

    def save(self, path, name):
        assert os.path.isdir(path) == True, \
            "Please provide a valid path. Path must be a directory."
        model_path = os.path.join(path, name)
        self.generator.save_weights(model_path)  # Load the generator
        return
    
    def load(self, path):
        assert os.path.isdir(path) == True, \
            "Please provide a valid path. Path must be a directory."
        self.generator = Generator(self.batch_size)
        self.generator = self.generator.load_weights(path)
        return self.generator
    
class Generator(tf.keras.Model):
    def __init__(self, batch_size):
        self.batch_size=batch_size
        
    def build_model(self, input_shape, dim, data_dim):
        input= Input(shape=input_shape, batch_size=self.batch_size)
        x = Dense(dim, activation='relu')(input)
        x = Dense(dim * 2, activation='relu')(x)
        x = Dense(dim * 4, activation='relu')(x)
        x = Dense(data_dim)(x)
        return Model(inputs=input, outputs=x)

class Discriminator(tf.keras.Model):
    def __init__(self,batch_size):
        self.batch_size=batch_size
    
    def build_model(self, input_shape, dim):
        input = Input(shape=input_shape, batch_size=self.batch_size)
        x = Dense(dim * 4, activation='relu')(input)
        x = Dropout(0.1)(x)
        x = Dense(dim * 2, activation='relu')(x)
        x = Dropout(0.1)(x)
        x = Dense(dim, activation='relu')(x)
        x = Dense(1, activation='sigmoid')(x)
        return Model(inputs=input, outputs=x)

In [12]:
data_cols = df.columns

In [13]:
#Define the GAN and training parameters
df[data_cols] = df[data_cols]

print(df.shape[1])

gan_args = [batch_size, learning_rate, noise_dim, df.shape[1], dim]
train_args = ['', epochs, log_step]

6


In [14]:
!mkdir modelmimic
!mkdir model/gan1
!mkdir model/gan1/savedmimic

mkdir: cannot create directory ‘modelmimic’: File exists
mkdir: cannot create directory ‘model/gan1’: No such file or directory
mkdir: cannot create directory ‘model/gan1/savedmimic’: No such file or directory


In [15]:
modelmimic = GAN
synthesizer = modelmimic(gan_args)
synthesizer.train(df, train_args)

0 [D loss: 0.748815, acc.: 25.00%] [G loss: 0.691517]


OSError: ignored